# Like fig. 7.7 but for emissions (plot is the remit of Sara / Terje / Sophie)

Theme Song: Deus Ex Machina<br>
Artist: Pure Reason Revolution<br>
Album: Amor Vincit Omnia<br>
Released: 2009

Temperature response to emissions 1750-2019

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
import matplotlib.patches as mp
from netCDF4 import Dataset
import warnings

from ar6.utils.h5 import *

In [2]:
results = load_dict_from_hdf5('../data_output_large/twolayer_AR6-historical-emissions-based.h5')

In [3]:
results.keys()

dict_keys(['AR6-anthro_climuncert', 'remove_blc_aci_climuncert', 'remove_blc_ari_climuncert', 'remove_blc_bcs_climuncert', 'remove_ch4_aci_climuncert', 'remove_ch4_ari_climuncert', 'remove_ch4_ch4_climuncert', 'remove_ch4_co2_climuncert', 'remove_ch4_h2o_climuncert', 'remove_ch4_ozo_climuncert', 'remove_co2_co2_climuncert', 'remove_con_con_climuncert', 'remove_luc_luc_climuncert', 'remove_n2o_aci_climuncert', 'remove_n2o_ari_climuncert', 'remove_n2o_ch4_climuncert', 'remove_n2o_n2o_climuncert', 'remove_n2o_ozo_climuncert', 'remove_nh3_ari_climuncert', 'remove_nox_aci_climuncert', 'remove_nox_ari_climuncert', 'remove_nox_ch4_climuncert', 'remove_nox_ozo_climuncert', 'remove_orc_aci_climuncert', 'remove_orc_ari_climuncert', 'remove_oth_aci_climuncert', 'remove_oth_ari_climuncert', 'remove_oth_ch4_climuncert', 'remove_oth_co2_climuncert', 'remove_oth_oth_climuncert', 'remove_oth_ozo_climuncert', 'remove_so2_aci_climuncert', 'remove_so2_ari_climuncert', 'remove_voc_aci_climuncert', 'remove

In [4]:
results['AR6-anthro_climuncert']['surface_temperature'].shape

(270, 2237)

In [5]:
results['AR6-anthro_climuncert']['surface_temperature'][0].mean()

0.0

In [6]:
forcings = list(results.keys())
forcings.remove('AR6-anthro_climuncert')
forcings

['remove_blc_aci_climuncert',
 'remove_blc_ari_climuncert',
 'remove_blc_bcs_climuncert',
 'remove_ch4_aci_climuncert',
 'remove_ch4_ari_climuncert',
 'remove_ch4_ch4_climuncert',
 'remove_ch4_co2_climuncert',
 'remove_ch4_h2o_climuncert',
 'remove_ch4_ozo_climuncert',
 'remove_co2_co2_climuncert',
 'remove_con_con_climuncert',
 'remove_luc_luc_climuncert',
 'remove_n2o_aci_climuncert',
 'remove_n2o_ari_climuncert',
 'remove_n2o_ch4_climuncert',
 'remove_n2o_n2o_climuncert',
 'remove_n2o_ozo_climuncert',
 'remove_nh3_ari_climuncert',
 'remove_nox_aci_climuncert',
 'remove_nox_ari_climuncert',
 'remove_nox_ch4_climuncert',
 'remove_nox_ozo_climuncert',
 'remove_orc_aci_climuncert',
 'remove_orc_ari_climuncert',
 'remove_oth_aci_climuncert',
 'remove_oth_ari_climuncert',
 'remove_oth_ch4_climuncert',
 'remove_oth_co2_climuncert',
 'remove_oth_oth_climuncert',
 'remove_oth_ozo_climuncert',
 'remove_so2_aci_climuncert',
 'remove_so2_ari_climuncert',
 'remove_voc_aci_climuncert',
 'remove_v

In [7]:
AR6_ecsforc = {}

for forcing in forcings:
    AR6_ecsforc[forcing[7:14]] = np.zeros(5)
    AR6_ecsforc[forcing[7:14]] = np.percentile(
        (results['AR6-anthro_climuncert']['surface_temperature'][-1] - results['AR6-anthro_climuncert']['surface_temperature'][0])-
        (results[forcing]['surface_temperature'][-1] - results[forcing]['surface_temperature'][0]), (5,16,50,84,95)
    )

In [8]:
AR6_ecsforc.keys()

dict_keys(['blc_aci', 'blc_ari', 'blc_bcs', 'ch4_aci', 'ch4_ari', 'ch4_ch4', 'ch4_co2', 'ch4_h2o', 'ch4_ozo', 'co2_co2', 'con_con', 'luc_luc', 'n2o_aci', 'n2o_ari', 'n2o_ch4', 'n2o_n2o', 'n2o_ozo', 'nh3_ari', 'nox_aci', 'nox_ari', 'nox_ch4', 'nox_ozo', 'orc_aci', 'orc_ari', 'oth_aci', 'oth_ari', 'oth_ch4', 'oth_co2', 'oth_oth', 'oth_ozo', 'so2_aci', 'so2_ari', 'voc_aci', 'voc_ari', 'voc_ch4', 'voc_co2', 'voc_ozo'])

In [9]:
print(AR6_ecsforc['ch4_co2'])

[0.00605408 0.00685012 0.00829478 0.01006824 0.01153   ]


In [10]:
emissions = ['co2', 'ch4', 'n2o', 'oth', 'nox', 'voc', 'so2', 'blc', 'orc', 'nh3', 'con', 'luc']
forc = ['co2', 'ch4', 'n2o', 'oth', 'ozo', 'h2o', 'ari', 'aci', 'bcs', 'con', 'luc']

emissions_full = ['CO2', 'CH4', 'N2O', 'Halocarbons', 'NOx', 'VOC', 'SO2', 'BC', 'OC', 'NH3', 'Contrails', 'Land use']
forcings_full = ['CO2', 'CH4', 'N2O', 'Halocarbons', 'O3', 'Stratospheric H2O', 'Aerosol-radiation', 'Aerosol-cloud', 'BC on snow', 'Contrails', 'Land use']

results = np.zeros((len(emissions), len(forc)))

for i, em in enumerate(emissions):
    for j, fo in enumerate(forc):
        combo = '%s_%s' % (em, fo)
        if combo in AR6_ecsforc.keys():
            #print(AR6_ecsforc[combo][2])
            results[i, j] = AR6_ecsforc[combo][2]

In [11]:
df50 = pd.DataFrame(results, index=emissions_full, columns=forcings_full)
df50.to_csv('../data_output/GSAT_by_emissions.csv')